Rui Wang 1007791695

# Bayesian Linear Regression:<br>Multivariate Normal Distributions [10 minutes]


\begin{align*}\require{cancel}
\mathbf{y}_{n\times 1} \sim {} & \mathcal{MVN}(\mathbf{X}_{n\times p}\boldsymbol{\beta}_{p\times 1},\boldsymbol \Sigma_{n\times n} = \sigma^2I_{n\times n})\\
p(\mathbf{y}|\boldsymbol \beta, \sigma, \mathbf{X}) = {}& (2\pi )^{-n/2}\cancel{\det({\boldsymbol {\Sigma }})^{-1/2}}{\frac{1}{\sigma^{\frac{n}{2}}}}\exp \left(-{\frac {1}{2\sigma^2}}(\mathbf {y} -\mathbf{X\boldsymbol\beta})^\top\cancel{\boldsymbol {\Sigma }^{-1}}(\mathbf {y} - \mathbf{X\boldsymbol\beta})\right)\\\\
\boldsymbol\beta \sim {} & \mathcal{MVN}(\boldsymbol\beta_0, \boldsymbol\Sigma_\beta) \quad or\;?\\\\
\sigma \sim {} & exponential(\lambda)  \quad p(\sigma) =  \lambda e^{-\lambda \sigma}1_{[0,\infty]}(\sigma)\\\\
\end{align*}
$$or \quad \sigma \sim \text{HalfNormal}(\mu_\sigma,\sigma_\sigma), \sigma \sim  \text{InverseGamma}(\alpha,\beta), \sigma \sim \text{TruncatedNormal}(\mu_\sigma,\sigma_\sigma,a,b) \quad or \; ?$$

In [ ]:
import pymc as pm; import numpy as np; n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # it's just a half normal, actually
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)
    # y = pm.MvNormal('y', mu=X@betas, cov=sigma**2*np.eye(n), shape=(n,1), observed=y)

with MLR:
    idata = pm.sample()

# Homework 5: Part I

1. Go get data from kaggle.com and do a ***Bayesian Linear Regression*** analysis

```python
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()
```    

2. Choose ***prior*** that are sensible: certainly you might change the ***hyperparameters***, and perhaps you can experiment with different distributional families for `sigma`...

3. [Optional] Assess the performance of the MCMC and note any issues or warnings

    1. Traceplots, inference (credible) intervals, effective sample sizes, energy plots, warnings and other notes... just the usual stuff they do [here](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

4. [Optional] Perform ***Multiple Linear Regression*** diagnostics... residual plots, etc.

# Part 0 #

I have found this dataset about Clothes Price Prediction in Kaggle, and here is the link to the dataset: https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset

This dataset contains 1338 rows of insured data, where the Insurance charges are given against the following attributes of the insured: Age, Sex, BMI, Number of Children, Smoker and Region. The attributes are a mix of numeric and categorical variables.

PS: I used this data set from HW3

In [ ]:
import random
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import requests

df = pd.read_csv("insurance.csv")

df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


I choose age as major parameter.

# Part 1 #


1.

In [ ]:
import pymc as pm; import numpy as np
n, p = 1338, 10; X, y = np.zeros((n,p)), np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu = np.zeros((p,1)), cov = np.eye(p), shape = (p,1))
    sigma = pm.TruncatedNormal('sigma', mu = 39, sigma = 14, lower = 0) # half normal
    y = pm.Normal('y', mu = pm.math.dot(X, betas), sigma = sigma, observed = y)

with MLR:
    idata = pm.sample()

2.

In [ ]:
import pymc as pm; import numpy as np
n, p = 1338, 39; X, y = np.zeros((n,p)), np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu = np.zeros((p,1)), cov = np.eye(p), shape = (p,1))
    sigma = pm.TruncatedNormal('sigma', mu = 39, sigma = 14, lower = 0) # half normal
    y = pm.Normal('y', mu = pm.math.dot(X, betas), sigma = sigma, observed = y)

with MLR:
    idata = pm.sample()

# Part 2 #

# Bayesian Linear Regression?<br>Geneal $\boldsymbol \Sigma$ Instead of $\sigma^2I$ [10 minutes]

\begin{align*}\require{cancel}
\mathbf{y}_{n\times 1} \sim {} & \mathcal{MVN}(\mathbf{X}_{n\times p}\boldsymbol{\beta}_{p\times 1},\boldsymbol \Sigma_{n\times n} = \sigma^2I_{n\times n})\\
p(\mathbf{y}|\boldsymbol \beta, \sigma, \mathbf{X}) = {}& (2\pi )^{-n/2}\cancel{\det({\boldsymbol {\Sigma }})^{-1/2}}{\frac{1}{\sigma^{\frac{n}{2}}}}\exp \left(-{\frac {1}{2\sigma^2}}(\mathbf {y} -\mathbf{X\boldsymbol\beta})^\top\cancel{\boldsymbol {\Sigma }^{-1}}(\mathbf {y} - \mathbf{X\boldsymbol\beta})\right)\\\hline\\
\mathbf{y}_{n\times 1} \sim {} & \mathcal{MVN}(\mathbf{X}_{n\times p}\boldsymbol{\beta}_{p\times 1},\boldsymbol \Sigma_{n\times n} \cancel{= \sigma^2I_{n\times n}})\\
p(\mathbf{y}|\boldsymbol \beta, \sigma, \mathbf{X}) = {}& (2\pi )^{-n/2}{\det({\boldsymbol {\Sigma }})^{-1/2}}\exp \left(-{\frac {1}{2}}(\mathbf {y} -\mathbf{X\boldsymbol\beta})^\top{\boldsymbol {\Sigma }^{-1}}(\mathbf {y} - \mathbf{X\boldsymbol\beta})\right)\\\\
\boldsymbol\beta \sim {} & \mathcal{MVN}(\boldsymbol\beta_0, \boldsymbol\Sigma_\beta) \quad or\;?\\\\
p(\boldsymbol \Sigma) \propto {} & \det({\boldsymbol {\Sigma }})^{-v/2} \exp \left(-{\frac {1}{2}}\textrm{tr}\left(\Psi{\boldsymbol {\Sigma }}^{-1} \right)\right) \quad or\;?
\end{align*}

## Do we forsee any problems with what we're going to do here?

# Conjugate Multivariate Normal Priors [15 minutes]

\begin{align*}
p(\mathbf{y} | \boldsymbol \beta, \boldsymbol \Sigma, \mathbf{X})= {}& (2\pi )^{-n/2}\det({\boldsymbol {\Sigma }})^{-1/2}\,\exp \left(-{\frac {1}{2}}(\mathbf {y} -\mathbf{X\beta})^\top{\boldsymbol {\Sigma }}^{-1}(\mathbf{y} - \mathbf{X\beta})\right)\\
\propto{}& \exp\left(\mathbf{\beta}^\top\mathbf{X}^\top\boldsymbol\Sigma^{-1}\mathbf{y} - \frac{1}{2}\mathbf{\beta}^\top \mathbf{X}^\top \boldsymbol\Sigma^{-1} \mathbf{X\beta} \right)\\
\propto{}& \exp\left( - \frac{1}{2} \left( \beta - (\mathbf{X}^\top \boldsymbol\Sigma^{-1} \mathbf{X})^{-1}\mathbf{X}^\top \boldsymbol\Sigma^{-1}\mathbf{y}\right)^{\!\top} \left[\mathbf{X}^{\top} \boldsymbol\Sigma^{-1} \mathbf{X} \right]\left( \beta - (\mathbf{X}^\top \boldsymbol\Sigma^{-1} \mathbf{X})^{-1}\mathbf{X}^\top \boldsymbol\Sigma^{-1}\mathbf{y}\right)  \right)\\
p(\boldsymbol \beta) \propto 1 \Longrightarrow {}& p(\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}) = \mathcal{MVN}\left(E[\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X}^\top \boldsymbol\Sigma^{-1} \mathbf{X})^{-1}\mathbf{X}^\top \boldsymbol\Sigma^{-1}\mathbf{y}, \text{Cov}[\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \left[\mathbf{X}^{\top} \boldsymbol\Sigma^{-1} \mathbf{X} \right]^{-1} \right)\\
or \quad {}& \text{to use a conjugate family of priors...}\\
p(\boldsymbol \beta) = {}& \mathcal{MVN}\left(E[\boldsymbol \beta ] = \boldsymbol \beta_0, \text{Cov}[\boldsymbol \beta ] = \boldsymbol \Sigma_\beta \right) \quad \text{ so}\\
p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}) = {}& \mathcal{MVN}\left(\overset{E[\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}] \; = \quad\quad}{\text{Cov}[\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}]}\bigg( \mathbf{X}^\top \boldsymbol\Sigma^{-1}\mathbf{y} + \boldsymbol \Sigma_\beta^{-1}\boldsymbol\beta_0 \right), \text{Cov}[\boldsymbol \beta | \boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \left[\mathbf{X}^{\top}\boldsymbol \Sigma^{-1} \mathbf{X}  + \boldsymbol \Sigma_\beta^{-1} \right]^{-1} \bigg)\\
\end{align*}


\begin{align*}
\text{Look familiar?}&\, \Downarrow{} \quad\quad\quad \underbrace{\;\,\textbf{mean}\,\;}\quad\;\underbrace{\textbf{precision}}\\
p(\theta|x,\theta_0,\tau, \phi) &={} \text{N}\left(\frac{\left(\tau \theta_0+\phi\sum_{i=1}^{n}x_{i}\right)}{(\tau+n\phi)}, \tau+n\phi \right)
\end{align*}

# Homework 5: Part II
    
## Answer the following with respect to $p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y})$ on the previous slide
    
1. Rewrite $p(\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y})$ in terms of $\sigma^2$ (no longer using $\Sigma$) if $\Sigma=\sigma^2I$

2. What is $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

3. What ***hyperparameters*** values (legal or illegal) would make $E[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = (\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

4. What ***hyperparameters*** values (legal or illegal) would make $E[  \mathbf{\hat y} = \mathbf{X}\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}] = \mathbf{X}(\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$?

5. What is $\text{Var}[\boldsymbol \beta |\boldsymbol\Sigma, \mathbf{X},\mathbf{y}]$?

1.

$p(\boldsymbol{\beta} | \sigma^2, \mathbf{X}, \mathbf{y}) = \mathcal{MVN}\left( E[\boldsymbol{\beta} | \sigma^2, \mathbf{X}, \mathbf{y}] = \left( \frac{1}{\sigma^2} \mathbf{X}^\top \mathbf{y} + \sigma_\beta^2 \boldsymbol{\Sigma}_\beta^{-1} \boldsymbol{\beta}_0 \right), \text{Cov}[\boldsymbol{\beta} | \sigma^2, \mathbf{X}, \mathbf{y}] = \frac{\sigma^2}{\mathbf{X}^\top \mathbf{X} + \sigma_\beta^2 \boldsymbol{\Sigma}_\beta^{-1}} \right)$


2.

$E[\boldsymbol{\beta} | \boldsymbol{\Sigma}, \mathbf{X},\mathbf{y}] = \left( \frac{1}{\sigma^2} \mathbf{X}^\top \mathbf{y} + \sigma_\beta^2 \boldsymbol{\Sigma}_\beta^{-1} \boldsymbol{\beta}_0 \right)$

3.

To have $E[\boldsymbol{\beta} |\boldsymbol{\Sigma}, \mathbf{X},\mathbf{y}] = (\mathbf{X^\top X})^{-1}\mathbf{X^\top y}$, you would need the following hyperparameters:

1. Set $\boldsymbol{\beta}_0$ very close to zero.
2. Set ${\Sigma}_\beta$ such that it has minimal influence on the posterior distribution compared to the likelihood term, which implies a very small variance.

4.

Same answer in Q3

5.

$p(\text{Var}[\boldsymbol{\beta} | \boldsymbol{\Sigma}, \mathbf{X}, \mathbf{y}]) = \left[\mathbf{X}^{\top} \boldsymbol{\Sigma}^{-1} \mathbf{X} + \boldsymbol{\Sigma}_\beta^{-1} \right]^{-1}$

# Homework 5: Part III

1. Go get data from kaggle.com and perform inference for a ***Bayesian Multivariate Normal Model***

<SPAN STYLE="font-size:18.0pt">

```python
import numpy as np; from scipy import stats
p=10; Psi=np.eye(p); a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)
n=1000; y=stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)
# Replace this made up data with your data set from kaggle...
    
with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p);
    # y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
with MNV_LKJ    
    idata = pm.sample()
```    
</SPAN>

2. As indicated above, don't use a covariance matrix parameterization and instead figure out how to parameterize this with a ***Cholesky factor*** to improve computational efficiency. The ***Cholesky***-based formulation allows general $O(n^3)$ $\det({\boldsymbol \Sigma})$ to be computed using a simple $O(n)$ product and general $O(n^3)$ ${\boldsymbol \Sigma}^{-1}$ to be instead evaluated with $O(n^2)$ ***backward substitution***.

2. Specify ***priors*** that work: certainly you'll likely need to change the ***prior hyperparameters*** for $\boldsymbol \mu$ (`mu`) and $\mathbf{R}$ (`packed_L`)...
    1. And you could consider adjusting the ***prior*** for $\boldsymbol \sigma$ using `sd_dist`...

3. [Optional] Assess the performance of the MCMC and note any issues

    1. Traceplots, inference (credible) intervals, effective sample sizes, energy plots, warnings and other notes... just the usual stuff they do [here](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

In [ ]:
import pymc as pm
import numpy as np
from scipy import stats

p = 39
Psi = np.eye(p)
a_cov = stats.invwishart(df=p+2, scale=Psi).rvs(1)

n = 1338
y = stats.multivariate_normal(mean=np.zeros(p), cov=a_cov).rvs(size=n)


with pm.Model() as MNV_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=p), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    Sigma = pm.Deterministic('Sigma', L.dot(L.T))
    mu = pm.MvNormal('mu', mu=np.zeros(p), cov=np.eye(p), shape=p)
    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

with MNV_LKJ:
    trace = pm.sample(1338)